In [1]:
import pandas as pd
import requests
from sqlalchemy import create_engine, text
import json

base_url = 'https://raw.githubusercontent.com/Shyamalasridar/exa-data-eng-assessment/main/data/'
json_files = ['Bill567_Breitenberg711_1029f880-d3db-f477-9da3-f59c14ed22c6.json']

# Extracting common patient ID
common_patient_id = None
for json_file in json_files:
    response = requests.get(base_url + json_file)
    data = response.json()
    for entry in data['entry']:
        resource = entry['resource']
        if resource['resourceType'] == 'Patient':
            common_patient_id = resource['id']
            break
    if common_patient_id:
        break



patient_data_list = []
condition_data_list = []
diagnostic_report_data_list = []
document_reference_data_list = []
claim_data_list = []
medication_request_data_list = []

for json_file in json_files:
    response = requests.get(base_url + json_file)
    data = response.json()
    for entry in data['entry']:
        resource = entry['resource']
        resource_type = resource['resourceType']
        full_url = entry['fullUrl']
        
        if resource_type == 'Patient':
            name = ''
            if 'name' in resource and resource['name']:
                name_parts = []
                for part in resource['name'][0]['given']:
                    name_parts.append(part)
                name_parts.append(resource['name'][0]['family'])
                name = ' '.join(name_parts)
                
            patient_data = {
                'fullUrl': entry['fullUrl'],
                'resourceType': resource['resourceType'],
                'id': resource['id'],
                'name': name,
                'gender': resource.get('gender', ''),
                'birthDate': resource.get('birthDate', ''),
                'deceasedDateTime': resource.get('deceasedDateTime', ''),
                'maritalStatus': resource.get('maritalStatus', {}).get('text', ''),
                'city': resource['address'][0].get('city', '') if 'address' in resource and resource['address'] else '',
                'state': resource['address'][0].get('state', '') if 'address' in resource and resource['address'] else '',
                'country': resource['address'][0].get('country', '') if 'address' in resource and resource['address'] else ''
            }
            patient_data_list.append(patient_data)
                       
        elif resource_type == 'Condition':
            condition = {
                'FullUrl': full_url,
                'ID': resource['id'],
                'Patient_ID': common_patient_id,  # Using common_patient_id here
                'Clinical_Status': resource.get('clinicalStatus', {}).get('coding', [{}])[0].get('display', ''),
                'Verification_Status': resource.get('verificationStatus', {}).get('coding', [{}])[0].get('display', ''),
                'Category': resource.get('category', [{}])[0].get('coding', [{}])[0].get('display', ''),
                'Code': resource.get('code', {}).get('coding', [{}])[0].get('display', '')
            }
            condition_data_list.append(condition)
        
        elif resource_type == 'DiagnosticReport':
            diagnostic_report = {
                'FullUrl': full_url,
                'ID': resource['id'],
                'Patient_ID': common_patient_id,  # Using common_patient_id here
                'Status': resource['status'],
                'Category': resource.get('category', [{}])[0].get('coding', [{}])[0].get('display', ''),
                'Code': resource['code']['coding'][0]['display'],
                'Performer': json.dumps(resource.get('performer', []))
            }
            diagnostic_report_data_list.append(diagnostic_report)
            
        elif resource_type == 'DocumentReference':
            document_reference = {
                'FullUrl': full_url,
                'ID': resource['id'],
                'Patient_ID': common_patient_id,  # Using common_patient_id here
                'Status': resource['status'],
                'Type': resource.get('type', {}).get('coding', [{}])[0].get('display', ''),
                'Category': resource.get('category', [{}])[0].get('coding', [{}])[0].get('display', ''),
                'Author': json.dumps(resource.get('author', [])),
                'Custodian': resource.get('custodian', {}).get('display', ''),
                'Content': json.dumps(resource.get('content', [])),
                'Context': json.dumps(resource.get('context', {}))
            }
            document_reference_data_list.append(document_reference)
        
        elif resource_type == 'Claim':
            billable_period = resource.get('billablePeriod', {})
            claim = {
                'ID': resource.get('patient', {}).get('reference', 'Unknown').split('/')[-1],  # Extracting reference ID from the patient field
                'Status': resource['status'],
                'Type': resource.get('type', {}).get('coding', [{}])[0].get('display', ''),
                'Patient_ID': common_patient_id,  # Using common_patient_id here
                'BillablePeriod_Start': billable_period.get('start'),
                'BillablePeriod_End': billable_period.get('end'),
                'Created': resource.get('created'),
                'Provider': resource.get('provider', {}).get('display', ''),
                'Priority': resource.get('priority', {}).get('coding', [{}])[0].get('display', ''),
                'Facility': resource.get('facility', {}).get('display', '')
            }
            claim_data_list.append(claim)
        
        elif resource_type == 'MedicationRequest':
            medication_request = {
                'FullUrl': full_url,
                'ID': resource['id'],
                'Patient_ID': common_patient_id,  # Using common_patient_id here
                'Status': resource['status'],
                'Intent': resource['intent'],
                'Medication': resource.get('medicationCodeableConcept', {}).get('coding', [{}])[0].get('display', ''),
                'Subject': resource.get('subject', {}).get('reference', ''),
                'Encounter': resource.get('encounter', {}).get('reference', ''),
                'AuthoredOn': resource.get('authoredOn', ''),
                'Requester': resource.get('requester', {}).get('display', ''),
                'DosageInstruction': json.dumps(resource.get('dosageInstruction', []))
            }
            medication_request_data_list.append(medication_request)

# Convert lists to DataFrames
patient_df = pd.DataFrame(patient_data_list)
condition_df = pd.DataFrame(condition_data_list)
diagnostic_report_df = pd.DataFrame(diagnostic_report_data_list)
document_reference_df = pd.DataFrame(document_reference_data_list)
claim_df = pd.DataFrame(claim_data_list)
medication_request_df = pd.DataFrame(medication_request_data_list)

# Database connection
server = 'AMSAS'
database = 'EMIS-Testing1'
connection_string = f'mssql+pyodbc://{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)

def delete_existing_records(engine, table_name, resource_id):
    with engine.connect() as connection:
        delete_query = text(f"DELETE FROM {table_name} WHERE ID = :id")
        connection.execute(delete_query, {'id': resource_id})

try:
    patient_table_name = 'PatientInformation'
    condition_table_name = 'Condition'
    diagnostic_report_table_name = 'DiagnosticReport'
    document_reference_table_name = 'DocumentReference'
    claim_table_name = 'Claim'
    medication_request_table_name = 'MedicationRequest'

    # Store PatientInformation data
    for index, row in patient_df.iterrows():
        delete_existing_records(engine, patient_table_name, row['id'])
        row.to_frame().T.to_sql(patient_table_name, engine, if_exists='append', index=False)

    # Store ConditionInformation data
    for index, row in condition_df.iterrows():
        delete_existing_records(engine, condition_table_name, row['ID'])
        row.to_frame().T.to_sql(condition_table_name, engine, if_exists='append', index=False)

    # Store DiagnosticReportInformation data
    for index, row in diagnostic_report_df.iterrows():
        delete_existing_records(engine, diagnostic_report_table_name, row['ID'])
        row.to_frame().T.to_sql(diagnostic_report_table_name, engine, if_exists='append', index=False)

    # Store DocumentReferenceInformation data
    for index, row in document_reference_df.iterrows():
        delete_existing_records(engine, document_reference_table_name, row['ID'])
        row.to_frame().T.to_sql(document_reference_table_name, engine, if_exists='append', index=False)

    # Store ClaimInformation data
    for index, row in claim_df.iterrows():
        delete_existing_records(engine, claim_table_name, row['ID'])
        row.to_frame().T.to_sql(claim_table_name, engine, if_exists='append', index=False)

    # Store MedicationRequestInformation data
    for index, row in medication_request_df.iterrows():
        delete_existing_records(engine, medication_request_table_name, row['ID'])
        row.to_frame().T.to_sql(medication_request_table_name, engine, if_exists='append', index=False)

    print(f"Data has been successfully saved to the SQL Server database '{database}', tables '{patient_table_name}', '{condition_table_name}', '{diagnostic_report_table_name}', '{document_reference_table_name}', '{claim_table_name}', and '{medication_request_table_name}'.")
except Exception as e:
    print(f"Error saving data to SQL Server: {e}")
finally:
    engine.dispose()


Data has been successfully saved to the SQL Server database 'EMIS-Testing1', tables 'PatientInformation', 'Condition', 'DiagnosticReport', 'DocumentReference', 'Claim', and 'MedicationRequest'.
